### Installing Packages
For this lectorial, we will investigate various control strategies for controlling articulated rigid body systems. This example set is written in Python 3, presented as a Jupyter notebook.

We will be making strong use of the open-source rigid-body dynamics library [Pinocchio](https://github.com/stack-of-tasks/pinocchio), as it will allow us to efficiently compute a number of kinodynamic expressions related to robotics and control.

To install Pinocchio in Python, we will be using `pip`, and it can be installed with the command
```python
pip install pin
```

We will also install a set of example robotics systems (containing URDF files and their accompanying visual and geometric files)

```python
pip install example-robot-data
```

And for visualisation of these models, the visualisation library Meshcat

```python
pip install meshcat
```

## Starting Example
Here we will load the manipulator [Panda](https://robodk.com/robot/Franka/Emika-Panda) and load it into the Meshcat visualiser with a desired configuration. You should see a new window open in your browser displaying the Panda arm in its default state.

In [16]:
import pinocchio as pin
import numpy as np
import time

from example_robot_data import load

robot = load('panda')

# Select the default configuration for the robot.
q0 = robot.q0
# OPTIONAL - Instead, choose a random configuration within the configuration space of the model
# q0 = pin.randomConfiguration(robot.model)

## visualise the robot
import meshcat
from pinocchio.visualize import MeshcatVisualizer
viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
# Start a new MeshCat server and client.
viz.initViewer(open=True)
# Load the robot in the viewer.
viz.loadViewerModel()
# Display the robot with the initial state q0
viz.display(q0)

# Display reference as a transparent red model
viz_ref = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz_ref.initViewer(viz.viewer)
viz_ref.loadViewerModel(rootNodeName="referemce", color=[1.0, 0.0, 0.0, 0.1])

# small time window for loading the model 
# if meshcat does not visualise the robot properly, augment the time
# it can be removed in most cases
time.sleep(0.2) 

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/


# Simulating the Dynamics Without Control
We will now use the rigid-body dynamics algorithms to forward-compute the dynamics of the robot from an initial state. When there is no joint stiffness or damping, the system will behave chaotically.

We are not accounting for collisions in this tutorial, as this would require a sophisticated physic simulator to handle the dynamics of contact, Meshcat is simply displaying the configuration of the robot.

In [30]:

def reference_generator(robot, t):
    qr = robot.q0
    # Provide sinusoidal generation to select joints
    qr[0] = np.sin(t)
    qr[2] = np.sin(t)
    qr[3] = np.sin(t)
    
    vr = np.zeros(robot.nv)
    
    return (qr, vr)


# Control inputs to each joint of the robot (set to zero) 
u = np.zeros(robot.nq)

# This is a very basic forward-simulator of the dynamics of the system given a control law controller, where the dynamics are forward-stepped by a first-order approximation.
def simulate(T, dt, robot, viz, viz_ref, controller, fps = 30):
    frame_cnt = 0

    # nq - Dimension of the robots configuration vector q
    # nv - Dimension of the robots velocity and acceleration vectors (v, a)
    
    q = robot.q0 # Configuration vector q
    v = np.zeros(robot.nv) # Velocity vector v
    a = np.zeros(robot.nv) # Acceleration vector a

    # Determine number of steps to wait until next frame
    n_skip = int((1.0 / fps) / dt)

    t = 0
    while t <= T:
        # Compute controller for current instanct
        u = controller(robot, t, q, v)
        # Calculate the current acceleration of the robot given the configuration, velocity and input
        a = pin.aba(robot.model, robot.data, q, v, u)
        # Forward step
        v = v + dt * a
        q = q + dt * v
        
        t += dt
        
        # Display every 24 frames
        if frame_cnt % n_skip == 0:
            viz.display(q)
            viz_ref.display(robot.q0)
            time.sleep(1.0 / fps)
        
        frame_cnt += 1

Let's start with simulating the system with no control input

In [18]:
def null_control(robot, t, q, v):
    return np.zeros(robot.nv)

# Simulator step resolution (s)
dt = 0.001

# Simulate the system for 5 seconds
simulate(5.0, dt, robot, viz, viz_ref, null_control)

## PD Joint Control
To track a nominal state, we can introduce virtual spring-damping to each joint by using a PD controller to act at each joint proportional to the error and error rate at each time step, much like in Lecture 1. 

<b>Note :</b> We are not using integral control here for convenience.

In [32]:
# Control Law u = Kp (q - qr) + Kd (v - vr)
def pd_controller(robot, t, q, v):
    # Define a reference
    qr, vr = reference_generator(robot, t)
    # Define gains
    Kp = np.diag([20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0])
    Kd = np.diag([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

    return  Kp.dot(qr - q) + Kd.dot(vr - v)

# Simulate the system for 5 seconds
simulate(5.0, dt, robot, viz, viz_ref, pd_controller)

## Computed Torque Control
That wasn't very good, was it? It looks as though our controller doesn't know the robot arm has weight, which is exactly what is happening. Let us instead view the controller from an acceleration perspective, if we can compute torques that achieve a desired acceleration in the joints, we can make this desired acceleration our error terms. We can then choose accelerations that are dynamically consistent with the robot, such that they also account for the dynamics of the robot.

In [33]:

def computed_torque_controller(robot, t, q, v):
    # Define a reference
    qr, vr = reference_generator(robot, t)
    ar = np.zeros(robot.nv)

    Kp = np.diag([20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0])
    Kd = np.diag([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

    # Set desired acceleration of the system as the PD error of the state
    ad = ar + Kp.dot(qr - q) + Kd.dot(vr - v)
    # Compute control inputs that would realise this acceleration on the model, accounting for its dynamics
    u = pin.rnea(robot.model, robot.data, q, v, ad)
    
    return  u

# Simulate the system for 5 seconds
simulate(5.0, dt, robot, viz, viz_ref, computed_torque_controller)

As you can see, the dynamics are now accounted for in the controller, thus the robot hardly moves. It should be noted this is a perfect simulation, where we know the dynamics of the robot exactly, if the models differ significantly, the behaviour may grow unstable.

## Operational Space Control
Moving the arm to any arbitrary point in space can be difficult from a computed torque perspective, as we need to know what joint configuration is required. 

Operational space control allows us to specify tasks that the robot needs to perform, and computes a torque to achieve it. This can range from positional tasks and impedance control (specify a desired force or spring-damper-like behaviour) to more abstract goals such as stability criterion and centre of mass tracking.

Continuing with our Panda example, we will make the robot follow a specified trajectory in space using operational space control.

In [ ]:
# Include symbolic algebra package Casadi
from casadi import *

def operational_space_control(robot, q, v):
    # Compute the positions of all bodies in the robot by using forward kinematics
    pin.forwardKinematics(robot.model, robot.data, q)
    # Joint ID
    JOINT_ID = 8

    # Compute task-space acceleration
    M = pin.crba(robot.model, robot.data, q)
    h = pin.rnea(robot.model, robot.data, q, v, np.zeros(robot.nq))
    
    # Track the position of the tip of the robot
    x = robot.data.oMi[JOINT_ID].translation
    # Get the Jacobian of the position in space
    J = pin.computeJointJacobian(robot.model, robot.data, q, JOINT_ID)[3:6, :]
    # Get the time-rate of change of the derivative
    pin.computeJointJacobiansTimeVariation(robot.model, robot.data, q, v)
    dJ = pin.getJointJacobianTimeVariation(robot.model, robot.data, JOINT_ID, pin.LOCAL)



    # Compute task-space control


    return np.zeros(robot.nv)

simulate(5.0, dt, robot, viz, operational_space_control)


## Model Predictive Control
